<a href="https://colab.research.google.com/github/andradeM17/Data-quality/blob/main/LLMs-as-judges/Round%202/gpt-oss/run-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openai/openai-cookbook/blob/main/articles/gpt-oss/run-colab.ipynb)

# Run OpenAI gpt-oss 20B in a FREE Google Colab

OpenAI released `gpt-oss` [120B](https://hf.co/openai/gpt-oss-120b) and [20B](https://hf.co/openai/gpt-oss-20b). Both models are Apache 2.0 licensed.

Specifically, `gpt-oss-20b` was made for lower latency and local or specialized use cases (21B parameters with 3.6B active parameters).

Since the models were trained in native MXFP4 quantization it makes it easy to run the 20B even in resource constrained environments like Google Colab.

Authored by: [Pedro](https://huggingface.co/pcuenq) and [VB](https://huggingface.co/reach-vb)

## Setup environment

In [1]:
!pip install -q --upgrade torch==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
!pip install -q transformers triton==3.4 kernels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip uninstall -q torchvision torchaudio -y

Please, restart your Colab runtime session after installing the packages above.

## Load the model and data

---



We load the model from here: [openai/gpt-oss-20b](https://hf.co/openai/gpt-oss-20b)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/411 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import pandas as pd

llm_data = pd.read_csv('/content/drive/MyDrive/llmj-data.csv')
print("llmdata.csv loaded successfully.")


llmdata.csv loaded successfully.


## Specify Reasoning Effort

Simply pass it as an additional argument to `apply_chat_template()`. Supported values are `"low"`, `"medium"` (default), or `"high"`.

In [ ]:
guidelines= '''You will be given a text.
If there are two samples, the first is English and the second is Irish. If the first sample is "Nan", the  second sample is Irish.
Using the guidelines, give the correct code for the text: NL, WL, X, CS, CB, or CC. Finish your statement in the format:
'
Annotation: [value]

';


Guidelines:
  IF the content is NOT linguistic OR you are unsure THEN annotate as NL NOTE: ""No or unsure"" → NL
  ELSE IF the text is parallel data AND the target side is NOT in Modern (standardised) Irish OR the source side is NOT in Modern English THEN annotate as WL NOTE: ""No or Unsure"" → WL NOTE: Untranslated named entities on either side are treated as belonging to that language (e.g., ""Capnat"" on the English side counts as English) NOTE: If text is entirely named entities, it is more likely to be ‘Yes’
  ELSE IF the Irish target text is NOT a direct translation of the English source text THEN annotate as X NOTE: ""Yes or unsure"" → next question; if not, annotate X
  ELSE IF the Irish text is short (just headings, single unrelated phrases, OR five words or fewer) THEN annotate as CS NOTE: Short text = CS
  ELSE IF the text is boilerplate OR low quality (text that would repeat across similar webpages, unnatural code-switching, unnatural phrasing, frequent unnatural formatting, misalignments) THEN annotate as CB NOTE: CB includes text not representative of natural language One formatting problem alone is NOT enough; more than one may qualify
  ELSE annotate as CC NOTE: CC = natural language, not short, not boilerplate, properly aligned.'''




for index, row in llm_data.iloc[1:4].iterrows(): # Skip the first row
    # Assuming the first two columns (index 0 and 1) contain the text
    # and need to be formatted as '"col1_content", "col2_content"'
    text = f'"{str(row.iloc[0])}", "{str(row.iloc[1])}"'
    print(f"\n{text}\n\n")

    messages = [
        {"role": "system", "content": guidelines},
        {"role": "user", "content": text},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort="low",
    ).to(model.device)

    generated = model.generate(**inputs, max_new_tokens=500)
    print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))


"nan", "

Eachtraí

Breitheanna

Básanna 

Blianta"


